In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import prune
from sklearn.model_selection import KFold
import pandas as pd
from sentence_transformers import SentenceTransformer

In [29]:
# Updated neural network for input/output size
class NeuralNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, output_dim)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_network(model, criterion, optimizer, X_train, Y_train, epochs=10):
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, Y_train)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item():.4f}")
    
    # Evaluate R^2 (coefficient of determination) at the end of training
    with torch.no_grad():
        model.eval()
        predictions = model(X_train)
        ss_total = ((Y_train - Y_train.mean()) ** 2).sum()
        ss_residual = ((Y_train - predictions) ** 2).sum()
        r2 = 1 - (ss_residual / ss_total)
        print(f"Final Training R^2: {r2.item():.4f}")

def determine_lambda(X, Y, model, criterion, k=5):
    # K-fold cross-validation to find the best lambda
    kfold = KFold(n_splits=k)
    lambdas = [0.01, 0.1, 1, 10]  # Example lambda values
    best_lambda = None
    best_score = float('inf')
    
    for lmbda in lambdas:
        print("\n",lmbda)
        scores = []
        for train_idx, val_idx in kfold.split(X):
            model_copy = NeuralNet(X.size(1), Y.size(1))  # Reset model
            optimizer = optim.Adam(model_copy.parameters())
            
            X_train, X_val = X[train_idx], X[val_idx]
            Y_train, Y_val = Y[train_idx], Y[val_idx]
            
            train_network(model_copy, criterion, optimizer, X_train, Y_train)
            val_outputs = model_copy(X_val)
            val_loss = criterion(val_outputs, Y_val).item()
            scores.append(val_loss + lmbda * torch.norm(model_copy.fc1.weight, p=1).item())
        
        avg_score = sum(scores) / len(scores)
        if avg_score < best_score:
            best_score = avg_score
            best_lambda = lmbda
    
    return best_lambda

def prune_model(model, lambda_value):
    # Apply L1 magnitude-based pruning
    parameters_to_prune = [(model.fc1, 'weight'), (model.fc2, 'weight')]
    prune.global_unstructured(
        parameters_to_prune,
        pruning_method=prune.L1Unstructured,
        amount=lambda_value,
    )
    return model

# Algorithm implementation
def iterative_pruning(X, Y, input_dim, output_dim, termination_condition, iterations):
    X, Y = torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)
    model = NeuralNet(input_dim, output_dim)
    criterion = nn.MSELoss()  # Example loss function
    optimizer = optim.Adam(model.parameters())
    i = 0
    
    while True:
        # Step 1: Train or retrain the neural network
        train_network(model, criterion, optimizer, X, Y)
        
        # Step 2: Introduce magnitude parameter β and prune
        lambda_value = determine_lambda(X, Y, model, criterion)
        model = prune_model(model, lambda_value)
        
        # Step 3: Check termination condition
        remaining_weights = sum(torch.sum(torch.abs(p) > 0).item() for p in model.parameters())
        if termination_condition(remaining_weights) or i==iterations:
            break
        
        # Step 4: Prune variables (features) with β_i = 0
        non_zero_features = (model.fc1.weight.abs().sum(dim=0) > 0).nonzero(as_tuple=True)[0]
        X = X[:, non_zero_features]
        model = NeuralNet(X.size(1), output_dim)  # Reinitialize model with remaining features

        i+=1
    
    return model

# Example usage
def termination_condition(remaining_weights, threshold=50000):
    print(remaining_weights)
    return remaining_weights < threshold

In [26]:
df = pd.read_csv('https://raw.githubusercontent.com/tiagoft/NLP/main/wiki_movie_plots_drama_comedy.csv')

x, y = df["Plot"][0:1000], df["Genre"][0:1000]

# df = pd.read_csv('pumadyn-32nm\puma32H.data', sep=',', header=None)

# x, y = df.iloc[:, :-1], df.iloc[:, -1]

display(df.head())

,Plot,Genre
0,The film is about a family who move to the sub...,comedy
1,Before heading out to a baseball game at a nea...,comedy
2,The plot is that of a black woman going to the...,comedy
3,On a beautiful summer day a father and mother ...,drama
4,A thug accosts a girl as she leaves her workpl...,drama


In [27]:
print("Loading Sentence Transformer model...")
model = SentenceTransformer("all-MiniLM-L6-v2")

x_embeddings = model.encode(x.tolist(), convert_to_tensor=True)
y_embeddings = model.encode(y.tolist(), convert_to_tensor=True)

# x_embeddings = torch.tensor(x.values).float()
# y_embeddings = torch.tensor(y.values).float()
# y_embeddings = y_embeddings.view(-1, 1)

Loading Sentence Transformer model...


In [28]:
print(x_embeddings.shape, y_embeddings.shape)

torch.Size([1000, 384]) torch.Size([1000, 384])


In [30]:
final_model = iterative_pruning(x_embeddings, y_embeddings, input_dim=x_embeddings.shape[1], output_dim=x_embeddings.shape[1], termination_condition=lambda w: termination_condition(w), iterations=10)


C:\Users\Ana Carolina Souza\AppData\Local\Temp\ipykernel_26796\3695924853.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X, Y = torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)


Epoch 1/10, Loss: 0.0042
Epoch 2/10, Loss: 0.0036
Epoch 3/10, Loss: 0.0030
Epoch 4/10, Loss: 0.0025
Epoch 5/10, Loss: 0.0021
Epoch 6/10, Loss: 0.0017
Epoch 7/10, Loss: 0.0014
Epoch 8/10, Loss: 0.0012
Epoch 9/10, Loss: 0.0011
Epoch 10/10, Loss: 0.0011
Final Training R^2: 0.6183

 0.01
Epoch 1/10, Loss: 0.0041
Epoch 2/10, Loss: 0.0034
Epoch 3/10, Loss: 0.0029
Epoch 4/10, Loss: 0.0023
Epoch 5/10, Loss: 0.0019
Epoch 6/10, Loss: 0.0015
Epoch 7/10, Loss: 0.0012
Epoch 8/10, Loss: 0.0011
Epoch 9/10, Loss: 0.0011
Epoch 10/10, Loss: 0.0010
Final Training R^2: 0.6316
Epoch 1/10, Loss: 0.0041
Epoch 2/10, Loss: 0.0034
Epoch 3/10, Loss: 0.0029
Epoch 4/10, Loss: 0.0024
Epoch 5/10, Loss: 0.0020
Epoch 6/10, Loss: 0.0016
Epoch 7/10, Loss: 0.0013
Epoch 8/10, Loss: 0.0011
Epoch 9/10, Loss: 0.0011
Epoch 10/10, Loss: 0.0010
Final Training R^2: 0.6391
Epoch 1/10, Loss: 0.0041
Epoch 2/10, Loss: 0.0034
Epoch 3/10, Loss: 0.0029
Epoch 4/10, Loss: 0.0025
Epoch 5/10, Loss: 0.0020
Epoch 6/10, Loss: 0.0017
Epoch 7/1